# An AI agent plays tic-tac-toe (part 1): building the opponent to play against
*Where we build a brute force minimax tree search which will serve as the opponent to our reinforcement learning approach*

*This article is part of a series that lets a computer play tic-tac-toe using reinforcement learning. The goal is to provide a complete implementation that you can really pick apart and learn reinforcement learning from. It is probably best to read the articles in order. The article including all the code [can be found on Github](https://github.com/PaulHiemstra/minmax_paper/blob/master/min_max_paper.ipynb).*

# What exactly is reinforcement learning? 
Reinforcement learning is the third modeling paradigm next to supervised and unsupervised modeling. It has seen great  successes recently, including [beating the worlds best Go player](https://deepmind.com/research/case-studies/alphago-the-story-so-far). This feat was considered impossible only a few year prior because of the enormous possible moveset in Go that dwarfs even that of chess. I will only briefly touch on how reinforcement learning works, but the following youtube movies should provide a more in depth introduction ([Intro RL](https://youtu.be/0MNVhXEX9to), [Deep RL](https://youtu.be/IUiKAD6cuTA)). 

Reinforcement learning (RL) is based on the following key concepts:

![tree](rl_flow.png)

There is an computer agent which takes actions (**A**) that act on environment (**E**). That environment responds by providing a reward (**R**) for that action, and bringing the system to the next state (**S**). Take for example the following state:

![tree](tictoe_board.png)

Here we consider two options for the next action of the O player: the green and red option. Taking the green option yields a high reward, as we block the X player from winning. Alternatively, the red option yields a low reward as this allows the X player to win. By playing tic-tac-toe many times and receiving rewards the RL agent will slowly learn what action in a particular state provides a good long term **value**. The mapping of state to optimal action is known as a **policy** in RL jargon. 

# Our first simple minimax algorithm
With the basics of RL out of the way, we can now focus on the goal of this article: building an opponent for our RL. Luckily for us, the game of tic-tac-toe is simple enough and can be solved in a brute force manner. This ensures that we end up with the optimal moves in tic-tac-toe, ensuring our RL agent can learn from the best. 

The main idea is to store all the possible tic-tac-toe states in a tree, and determine for a given state what next move leads to a good result. This is done by recursively searching through the tree and finding rewards. The reward in the subtree below each possible next move informs us which moves is optimal. To illustrate how this works, we start with a very simple tree ([based on this youtube video](https://www.youtube.com/watch?v=5oXyibEgJr0)): 

![tree](simple_minmax.png)

In this game two players compete: one has the goal to maximize the score, one has the goal to minimize the score. First the maximizing player can choose at the top of the tree, then the minimizing player can choose and so on. As the maximizing player our optimal move *seems* to be to go right, that is where the high score of 9 is present. However, the second player will then simply choose 2 when it has the choice. Therefore, in this case the optimal choice is to go left and get a score of three. 

Solving this kind of tree is done using a [minimax algorithm](https://en.wikipedia.org/wiki/Minimax). At each level in the tree the min or max player makes their optimal choice. If we express this tree in Python we get:

In [3]:
from treelib import Node, Tree

tree = Tree()
tree.create_node("root", "root")
tree.create_node("", "l1", parent='root')
tree.create_node("", "r1", parent='root')
tree.create_node("3", "l1-1", parent='l1')
tree.create_node("5", "l1-2", parent='l1')
tree.create_node("2", "r1-1", parent='r1')
tree.create_node("9", "r1-2", parent='r1')
tree
tree.show()

root
├── 
│   ├── 3
│   └── 5
└── 
    ├── 2
    └── 9



where we can solve the minimax problem by recusively going through the tree:

In [4]:
def minmax(tree, current_id, is_max):
    '''
    tree: tree object to minmax over
    current_id: current_id where we are in the tree
    is_max: are we the maximizing player?
    '''
    if tree.depth(current_id) == tree.depth():             # If this holds, we are at the end of the tree
        return int(tree[current_id].tag)                   # return the value at the end of tree so it propagates up the tree
    children_of_current_id = tree.children(current_id)     # Determine the children of the current node
    scores = [minmax(tree, child.identifier, not is_max) for child in children_of_current_id]   # Recursively run this function on each of the children
    if is_max:                                             # Return the appropriate score for the max or min player  
        return max(scores)
    else:
        return min(scores)

minmax(tree, 'root', True)

3

which indeed shows the correct maximum score the max player can get given this particular tree: 3. Making the recursion explicit in pseudo-code reveals what happens under the hood:
    
    start-game: minmax('root') -> 
    max-player: max([minmax('l1'), minmax('r1')]) -> 
    min-player: max([min([minmax('l1-1), minmax('l1-2')]), 
                     min([minmax('r1-1'), minmax('r1-1')])]) ->
    end-tree  : max([min([3, 5]), 
                     min([2, 9])]) -> 
                max([3, 2]) ->
                3

# Minmax tree search on a tic-tac-toe tree
We can use the exact same minimax approach for tic-tac-toe using a much larger tree. First we start at the top with nine possible moves for the max player, then we are left with eight moves for the min player, etcetera:

![tree](tictoe_tree.png)

which leaves us with an enormous tree with all the possible combinations possible on the board. To store the tic-tac-toe state we assign a letter toe each of the fields on the board:

![tree](tictoe_letters.png)

and create a Python class to store the state, allow updates to the board, check if the game is done, and return the value of our game depending on the outcome:

In [5]:
import numpy as np

class Tictoe:
    def __init__(self, size):
        self.size = size
        self.board = np.zeros(size*size)
        self.letters_to_move = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'][:size*size]
    def get_board(self):
        return self.board.reshape([self.size, self.size])
    def make_move(self, who, where, verbose=False):
        self.board[self.letters_to_move.index(where)] = who
    def get_sums_of_board(self):
        local_board = self.get_board()
        return np.concatenate([local_board.sum(axis=0),             # columns
                               local_board.sum(axis=1),             # rows
                               np.trace(local_board),               # diagonal
                               np.trace(np.fliplr(local_board))], axis=None)   # other diagonal
    def is_endstate(self):
        someone_won = len(np.intersect1d((self.size, -self.size), self.get_sums_of_board())) > 0
        draw = np.count_nonzero(self.board) == self.size * self.size
        return someone_won or draw
    def get_value(self):
        sums = self.get_sums_of_board()
        if self.size in sums:
            return 10
        elif -self.size in sums:
            return -10
        else:
            return 0

next we recursively create the giant tree and add the board state object at each node. The ID of each node is the sequence of steps taken, e.g. `acig`. 

In [ ]:
import copy

def remove_value_list(l, val):
    return [el for el in l if el != val]

flip_player = {1: -1, -1: 1}

possible_options = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

def add_options_to_node(tree, node, tt_data, player, remaining_options):
    for option in remaining_options:
        local_tt_data = copy.deepcopy(tt_data)           # To prevent changing these values in other branches of the tree
        local_tt_data.make_move(player, option, False)
        if node.identifier != 'root':
            new_identifier = node.identifier + option
        else:
            new_identifier = option
        tree.create_node(option, new_identifier, node.identifier, data = local_tt_data)
        if len(remaining_options) > 1 and not local_tt_data.is_endstate():  # At end of the game, stop adding nodes
            add_options_to_node(tree, tree[new_identifier], local_tt_data, 
                                flip_player[player], remove_value_list(remaining_options, option))
    return None

TicToe_state = Tictoe(3)
TicToe_3x3 = Tree()
TicToe_3x3.create_node("root", "root")
add_options_to_node(TicToe_3x3, TicToe_3x3["root"], 
                    TicToe_state, 1, possible_options)

With the tree done, we can use minimax code very similar to the simple tree we first started with:

In [7]:
def minmax_tt(tree, current_id, is_max):
    current_node = tree[current_id]                     # Find the tree element we are now
    if current_node.data.is_endstate():                 # Are we at the end of the game?
        return current_node.data.get_value()            # Return the value
    children_of_current_id = tree.children(current_id)  # Determine the children
    scores = [minmax_tt(tree, child.identifier, not is_max) for child in children_of_current_id]   # Recursively run this function on each of the children
    if is_max:                                          # Return the max or min score depending on which player we are
        return max(scores)
    else:
        return min(scores)
    
def determine_move(tree, current_id, is_max):
    '''
    Given a state on the board, what is the best next move? 
    '''
    potential_moves = tree.children(current_id)
    moves = [child.identifier[-1] for child in potential_moves]
    raw_scores = [minmax_tt(tree, child.identifier, not is_max) for child in potential_moves]
    if is_max:
        return moves[raw_scores.index(max(raw_scores))]
    else:
        return moves[raw_scores.index(min(raw_scores))]

Using `determine_move` we can now get the next best move. For example, when the max player plays the `a` square:

In [10]:
import time

start = time.time()
print(determine_move(TicToe_3x3, 'a', is_max=False))
print(time.time()-start)

e
2.986009120941162


Note that it picks the center `e` square, which is indeed a very strong move in tic-tac-toe. 

# Solving for slowness
So, now we have a worthy opponent for our RL agent. The only problem is that it is very slow, much too slow actually for our RL agent which will have to play a lot of games to learn tic-tac-toe. In part 2 we will use an advanced programming technique to massively speedup the tree search. 

In [ ]:
tictactoe = Tictoe(3)

print('''Welcome to TicTacToe. 

You can make a move by selecting one of the following letters:''')
print(np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).reshape(3,3))
print('''You start, the computer will take the next move

Initial board:''')

move_history = ''
while not tictactoe.is_endstate():
    player_move = input('Your move!: ')
    tictactoe.make_move(1, player_move)
    print(tictactoe.get_board())
    move_history += player_move
    if tictactoe.is_endstate():
        print('You won!...wait you won?????')
    
    print('Computer is thinking')
    computer_move = determine_move(TicToe_3x3, move_history, False)
    tictactoe.make_move(-1, computer_move)
    print(tictactoe.get_board())
    move_history += computer_move
    if tictactoe.is_endstate():
        print('Computer won!')
        
    if len(move_history) >= 8 and not tictactoe.is_endstate():
        print('Draw...')
        break